In [ ]:
f=open("zamato_data/complete2021-08-20.csv")
links=[]
for x in f:
    links.append(x.replace(",","").replace("\n",""))

In [ ]:
links

In [ ]:
import requests
from bs4 import BeautifulSoup
URL = "https://www.zomato.com/dubai"
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0'}
cookies = {"AWSALBTG":"Qk8nb58rGC6eH+LEHI6eIFTZmN8lGnpvX8Lo/RN4OD3+pCvEQtv1lFXCjN1VoBsJspTZDiBWij14aaR/aqm1oEnLwpHDg/O6Woal44WSscWOmiy6JcvlAr7B+jVos8iS8UHv4M95zqhYknLqnfJHgLeWYCg46MvAhGzJ6dS1f8rI","AWSALBTGCORS":"Qk8nb58rGC6eH+LEHI6eIFTZmN8lGnpvX8Lo/RN4OD3+pCvEQtv1lFXCjN1VoBsJspTZDiBWij14aaR/aqm1oEnLwpHDg/O6Woal44WSscWOmiy6JcvlAr7B+jVos8iS8UHv4M95zqhYknLqnfJHgLeWYCg46MvAhGzJ6dS1f8rI", "csrf":"126a31142b6385c404cc7454b02573ae","fbcity":"57","fre":"0","rd":"1380000" ,"zl":"en", "fbtrack":"5d9b4c8f54bc7710d5ef6399c97c5b65","ltv":"57" ,"lty":"city", "locus":"%7B%22addressId%22%3A0%2C%22lat%22%3A24.404153%2C%22lng%22%3A54.500334%2C%22cityId%22%3A57%2C%22ltv%22%3A57%2C%22lty%22%3A%22city%22%2C%22fetchFromGoogle%22%3Afalse%2C%22dszId%22%3A5567%2C%22fen%22%3A%22Abu+Dhabi%22%7D", "PHPSESSID":"850a3db1b2c821fd4a1940d9ded3c2c9" ,"expab":"1","session_id":"6625547f344b3-7372-4847-b43d-4c3dc0ab16b2", "dpr":"1", "G_ENABLED_IDPS":"google"}
page=requests.get(URL,headers=headers)
soup = BeautifulSoup(page.content, "html.parser")
soup_body=soup.find("body")

In [ ]:
import re
all_city_links=[]
zamoto_cityO=soup_body.find_all("div",class_="sc-bke1zw-0")
for x in zamoto_cityO[2].find_all("a"):
    all_city_links.append(x.get("href"))

In [ ]:
all_city_links_with_page=[]
for zx in all_city_links:
    for xx in range(1,507):
        link=zx+"?page="+str(xx)
#         if requests.get(link,headers=headers,cookies=cookies).status_code == 200:
        all_city_links_with_page.append(link)

In [ ]:
import json 
from csv import writer
from datetime import datetime
rest_data=[]
error_data=[]
def get_data(initial,final):
    print(initial)
    rest_file=open(f"zamato_data/final_resturent_zamato{str(datetime.now().date())}.csv",'a')
    writer_object = writer(rest_file)
    rest_file_error=open(f"zamato_data/error{str(datetime.now().date())}.csv",'a')
    complete_list_file=open(f"zamato_data/complete{str(datetime.now().date())}.csv",'a')
    writer_object_error = writer(rest_file_error)
    writer_complete_list_file = writer(complete_list_file)
    for pageO in all_city_links_with_page[initial:final]:
        writer_complete_list_file.writerow(pageO)
        try:
            city_pageO=requests.get(pageO,headers=headers,cookies=cookies)
            city_soupO = BeautifulSoup(city_pageO.content, "html.parser")
            city_soup_bodyO=city_soupO.find("body")
            resturent_data=city_soup_bodyO.find_all("div",class_="search-snippet-card")
            for rest in resturent_data:
                data=[]
                rest_titleimage=rest.find("a","feat-img").get("data-original").split("?")[0]
                rest_id=rest_titleimage.split("/")[-2]
                rest_mobile_number=rest.find("a",class_="item res-snippet-ph-info").get("data-phone-no-str")
                resturent_name=rest.find("a",class_="result-title").text.strip().split("-")[0].strip()
                resturent_link=rest.find("a",class_="result-title").get("href")
                
                resturent_dining_rating_value=rest.find_all("span",class_="rating-value")
                resturent_dining_rating_review_count=rest.find_all("span",class_="review-count")
                if len(resturent_dining_rating_value)>1 and len(resturent_dining_rating_review_count)>1:
                    try:
                        dining_rating=resturent_dining_rating_value[0].text.strip().replace("(","").replace(")","").replace("Reviews","").replace("(","")
                    except:
                        dining_rating=0
                    try:    
                        delivery_rating=resturent_dining_rating_value[1].text.strip().replace("(","").replace(")","").replace("Reviews","").replace("(","")
                    except:
                        delivery_rating=0
                    try:    
                        dining_rating_people_count=resturent_dining_rating_review_count[0].text.strip().replace("Reviews","").replace("(","").replace(")","")
                    except:
                        dining_rating_people_count=0
                    try:    
                        delivery_rating_people_count=resturent_dining_rating_review_count[1].text.strip().replace("(","").replace(")","")
                    except:
                        delivery_rating_people_count=0
                else:
                    try:
                        dining_rating=resturent_dining_rating_value[0].text.strip().replace("(","").replace(")","").replace("Reviews","").replace("(","")
                        delivery_rating=0
                        dining_rating_people_count=resturent_dining_rating_review_count[0].text
                        delivery_rating_people_count=0
                    except:
                        dining_rating=0
                        delivery_rating=0
                        dining_rating_people_count=0
                        delivery_rating_people_count=0
                        
                        
                resturent_in=rest.b.text
                resturent_address=rest.find("div",class_="search-result-address").text
                caous=[]
                caousine=rest.find("span",class_="nowrap")
                for x in caousine.find_all("a"):
                    caous.append(x.text)
                cuisine=",".join(caous)
                try:
                    costoftwo=rest.find("div",class_="res-cost").find_all("span")[-1].text.split(" ")[-1]
                except:
                    costoftwo=""
                try:    
                    hours = rest.find("div",class_="search-grid-right-text").text.strip()
                except:
                    hours=""
                try:    
                    feat=rest.find("div",class_="res-collections").find_all("a")    
                    feature_in=[]
                    for x in feat:
                        feature_in.append(x.text)
                    feature_=",".join(feature_in)
                except:
                    feature_=""
                try:
                    di=rest.find_all("div",class_="res-offers clearfix")
                    if len(di) != 0:
                        for x in di:
                            dis=x.find("span").text
                            if dis == "Promotions:":
                                promotion=x.find("div",class_="search-grid-right-text").text.strip()
                                tc=x.find("div",class_="search-grid-right-text").find("a").get("data-html").strip()
                                break
                            else:
                                promotion=""
                                tc=""
                    else:            
                        promotion=""
                        tc=""

                except:
                    promotion=""
                    tc=""
                try:
                    di=rest.find_all("div",class_="res-offers clearfix")
                    for x in di:
                        dis=x.find("span").text
                        if dis == "Discount:":
                            discount=x.find("a").text.strip()
                            break
                        else:
                            discount=""
                    if len(di) == 0:    
                            discount=""          
                except:
                    discount=""
                rest_page=requests.get(resturent_link,headers=headers)
                rest_soup = BeautifulSoup(rest_page.content, "html.parser")
                rest_soup_body=rest_soup.find("body")   
                try:
                    zamato_pro=rest.find("span",class_="new_pro_res_tag").text.strip()
                    if zamato_pro == "zomato pro":
                        offer=rest_soup_body.find_all("p")[3].text
                        description=rest_soup_body.find_all("p")[4].text
                    else:
                        description=rest_soup_body.find_all("p")[3].text
                        offer=""
#                     print(offer)
#                     print(description)
                except:
                    description=rest_soup_body.find_all("p")[3].text     
                    offer=""
#                 try:
                for x in range(10,17):
                    try:
                        latitude,longitude=rest_soup_body.find_all("a")[x].get("href").split("=")[-1].split(",")
                        break
                    except:
                        latitude=""
                        longitude=""
                        
                
#                 except:
#                     latitude=""
#                     longitude=""
                data.append(str(rest_id))
                data.append(resturent_name)
                data.append(cuisine)
                data.append(rest_mobile_number)
                data.append(resturent_in)
                data.append(str(dining_rating))
                data.append(str(dining_rating_people_count))
                data.append(str(delivery_rating))
                data.append(str(delivery_rating_people_count))
                data.append(resturent_address)
                data.append(str(costoftwo))
                data.append(hours)
                data.append(feature_)
                data.append(promotion)
                data.append(tc)
                data.append(discount)
                data.append(str(latitude))
                data.append(str(longitude))
                data.append(rest_titleimage)
                data.append(resturent_link)
                data.append(description)
                data.append(offer)
                writer_object.writerow(data)
                print(resturent_link)
                
        except:
            print(str(initial)+"error"+"-----"+str(len(rest_data)))
            print(pageO)
            writer_object_error.writerow([pageO])
    rest_file.close()
    rest_file_error.close()
    complete_list_file.close()

#         print(promo1)
#         print(discount)
#         print(tc)


In [ ]:
import multiprocessing
t1 = multiprocessing.Process(target=get_data, args=(0,500))
t2 = multiprocessing.Process(target=get_data, args=(500,1000))
t3 = multiprocessing.Process(target=get_data, args=(1000,1500))
t4 = multiprocessing.Process(target=get_data, args=(1500,2000))
t5 = multiprocessing.Process(target=get_data, args=(2000,2500))
t6 = multiprocessing.Process(target=get_data, args=(2500,3136))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()

In [ ]:
import multiprocessing
t1 = multiprocessing.Process(target=get_data, args=(0,250))
t11 = multiprocessing.Process(target=get_data, args=(250,500))
t111 = multiprocessing.Process(target=get_data, args=(500,750))
t1111 = multiprocessing.Process(target=get_data, args=(750,1000))
t2 = multiprocessing.Process(target=get_data, args=(1000,2500))
t22 = multiprocessing.Process(target=get_data, args=(1000,2000))
t222 = multiprocessing.Process(target=get_data, args=(2000,3000))
t3 = multiprocessing.Process(target=get_data, args=(3000,4000))
t4 = multiprocessing.Process(target=get_data, args=(4000,5000))
t5 = multiprocessing.Process(target=get_data, args=(5000,6000))
t55 = multiprocessing.Process(target=get_data, args=(6000,7000))
t555 = multiprocessing.Process(target=get_data, args=(7000,8000))
t6 = multiprocessing.Process(target=get_data, args=(8000,9000))
t66 = multiprocessing.Process(target=get_data, args=(9000,10000))
t7 = multiprocessing.Process(target=get_data, args=(10000,12000))
t8 = multiprocessing.Process(target=get_data, args=(12000,12500))
t88 = multiprocessing.Process(target=get_data, args=(12500,13000))
t9 = multiprocessing.Process(target=get_data, args=(13000,13500))
t99 = multiprocessing.Process(target=get_data, args=(13500,14000))
t10 = multiprocessing.Process(target=get_data, args=(14000,14500))
t100 = multiprocessing.Process(target=get_data, args=(14500,14700))
t1000 = multiprocessing.Process(target=get_data, args=(14700,15180))




# t2 = threading.Thread(target=get_data, args=(10,20))
t1.start()
t11.start()
t111.start()
t1111.start()
t2.start()
t22.start()
t222.start()
t3.start()
t4.start()
t5.start()
t55.start()
t555.start()
t6.start()
t66.start()
t7.start()
t8.start()
t88.start()
t9.start()
t99.start()
t10.start()
t100.start()
t1000.start()



# starting thread 2
# t2.start()

# wait until thread 1 is completely executed
t1.join()
t11.join()
t111.join()
t1111.join()
t2.join()
t22.join()
t222.join()
t3.join()
t4.join()
t5.join()
t55.join()
t555.join()
t6.join()
t66.join()
t7.join()
t8.join()
t88.join()
t9.join()
t99.join()
t10.join()
t100.join()
t1000.join()


# wait until thread 2 is completely executed
# t2.join()

# both threads completely executed
print("Done!")

In [ ]:
import requests
resturent_link="https://www.zomato.com/chaatbazaar"
des_page=requests.get(resturent_link,headers=headers,cookies=cookies)
des_pageO=BeautifulSoup(des_page.content,"html.parser")
des_pageO_body=des_pageO.find("body")
descriptions=des_pageO_body.find_all("script")
print(len(descriptions))
descriptions[0]

In [ ]:
columns=['Restaurant Id',"Restaurant name","Type of Restaurant","Mobile number","Restaurant In","Dining Review","Dining Review count","Delivery Review",'Delivery Review Count',"Address","Costoftwo","Hours","Feature","Promotion","tc","Discount","latitude","longitude","titleimage","Restaurant link"]
import pandas as pd
data=pd.DataFrame(rest_data,columns=columns).fillna("")

In [ ]:
data.to_excel("lati_and_long.xls")

In [ ]:
rest_file=open("lati_and_long.csv",'a')
rest_file.write("|||".join(columns)+"\n")
for x in data.values:
    rest_file.write("|||".join(x)+"\n")
rest_file.close()    

In [ ]:
from csv import writer
columns=['Restaurant Id',"Restaurant name","Type of Restaurant","Mobile number","Restaurant In","Dining Review","Dining Review count","Delivery Review",'Delivery Review Count',"Address","Costoftwo","Hours","Feature","Promotion","tc","Discount","latitude","longitude","titleimage","Restaurant link"]
rest_file=open("lati_and_long.csv",'a')
writer_object = writer(rest_file,delimiter = "|||")
writer_object.writerow(columns)
for x in rest_data:
    writer_object.writerow(x)
rest_file.close()    

In [ ]:
import requests
from bs4 import BeautifulSoup
URL = "https://www.zomato.com/chaatbazaar"
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0'}
cookies = {"AWSALBTG":"Qk8nb58rGC6eH+LEHI6eIFTZmN8lGnpvX8Lo/RN4OD3+pCvEQtv1lFXCjN1VoBsJspTZDiBWij14aaR/aqm1oEnLwpHDg/O6Woal44WSscWOmiy6JcvlAr7B+jVos8iS8UHv4M95zqhYknLqnfJHgLeWYCg46MvAhGzJ6dS1f8rI","AWSALBTGCORS":"Qk8nb58rGC6eH+LEHI6eIFTZmN8lGnpvX8Lo/RN4OD3+pCvEQtv1lFXCjN1VoBsJspTZDiBWij14aaR/aqm1oEnLwpHDg/O6Woal44WSscWOmiy6JcvlAr7B+jVos8iS8UHv4M95zqhYknLqnfJHgLeWYCg46MvAhGzJ6dS1f8rI", "csrf":"126a31142b6385c404cc7454b02573ae","fbcity":"57","fre":"0","rd":"1380000" ,"zl":"en", "fbtrack":"5d9b4c8f54bc7710d5ef6399c97c5b65","ltv":"57" ,"lty":"city", "locus":"%7B%22addressId%22%3A0%2C%22lat%22%3A24.404153%2C%22lng%22%3A54.500334%2C%22cityId%22%3A57%2C%22ltv%22%3A57%2C%22lty%22%3A%22city%22%2C%22fetchFromGoogle%22%3Afalse%2C%22dszId%22%3A5567%2C%22fen%22%3A%22Abu+Dhabi%22%7D", "PHPSESSID":"850a3db1b2c821fd4a1940d9ded3c2c9" ,"expab":"1","session_id":"6625547f344b3-7372-4847-b43d-4c3dc0ab16b2", "dpr":"1", "G_ENABLED_IDPS":"google"}
rest_page=requests.get(URL,headers=headers)
rest_soup = BeautifulSoup(rest_page.content, "html.parser")
rest_soup_body=rest_soup.find("body")

In [ ]:
print(rest_soup_body.find("section",class_="sc-ixltIz gYgmFw"))

In [ ]:
error_data

In [ ]:
len(all_city_links_with_page)

In [ ]:
total_page=len(all_city_links_with_page)
for x in range(total_page):
    if x % 100 == 0:
        print(x)

In [ ]:
new_sample_data=pd.DataFrame(rest_data)

In [ ]:
error_data

In [ ]:
import pandas as pd
data=pd.DataFrame(rest_data,columns=['Restaurant Id',"Restaurant name","Type of Restaurant","Mobile number","Restaurant In","Dining Review","Dining Review count","Delivery Review",'Delivery Review Count',"Address","Costoftwo","Hours","Feature","Promotion","tc","Discount","latitude","longitude","titleimage","Restaurant link"])

In [ ]:
data.to_csv("lati_and_long.csv")

In [ ]:
data.columns

In [ ]:
import pandas as pd
data=pd.read_csv("lati_and_long.csv")

In [ ]:
for x in data.values:
    print(len(x))

In [ ]:
from csv import reader
data=open("lati_and_long.csv","r")
csv_data=reader(data,delimiter="|||")

In [ ]:
for x in csv_data:
    print(len(x))

In [ ]:
data

In [ ]:
print("DS")

In [ ]:
f=open("lati_and_long.xls","r")

In [ ]:
for x in f.read().split("\n"):
    print(len(x.split("|||")))
    print(x.split("|||"))

In [ ]:
f

In [ ]:
pd.read_excel("lati_and_long.xls").fillna("").values

In [ ]:
import pandas as pd
city_data=pd.read_csv("all_city.csv")

In [ ]:
city_data.iloc[:3000,1:5].to_csv("data_0_3000.csv")
city_data.iloc[3000:6000,1:5].to_csv("data_3000_6000.csv")
city_data.iloc[6000:9000,1:5].to_csv("data_6000_9000.csv")
city_data.iloc[9000:12000,1:5].to_csv("data_9000_12000.csv")
city_data.iloc[12000:15000,1:5].to_csv("data_12000_15000.csv")
city_data.iloc[15000:18000,1:5].to_csv("data_15000_18000.csv")
city_data.iloc[18000:21000,1:5].to_csv("data_18000_21000.csv")
city_data.iloc[21000:,1:5].to_csv("data_21000_end.csv")

In [ ]:
import pandas as pd
data=pd.read_csv("nationlity.csv")

In [ ]:
data['nationality'].values

In [17]:
import pandas as pd
datalist1=pd.read_csv("zamato_data/final_resturent_zamato2021-08-20.csv")
datalist2=pd.read_csv("zamato_data/final_resturent_zamatofinal_resturent_zamato2021-08-20.csv")

In [21]:
datalist=list(datalist1.values)+list(datalist2.values)

In [24]:
data=pd.DataFrame(datalist)

In [31]:
import pandas as pd
d=pd.read_csv("zamato_data/final_resturent_zamato2021-08-20.csv")

In [32]:
columns=['Restaurant Id',"Restaurant name","Type of Restaurant","Mobile number","Restaurant In","Dining Review","Dining Review count","Delivery Review",'Delivery Review Count',"Address","Costoftwo","Hours","Feature","Promotion","tc","Discount","latitude","longitude","titleimage","Restaurant link","Decription","offer"]
dataupload=pd.DataFrame(d.values[30000:31000],columns=columns)

In [33]:
dataupload.to_excel("zamato_data/uploadzamato30000-31000.xls")

<ipython-input-33-e6149a04368f>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  dataupload.to_excel("zamato_data/uploadzamato30000-31000.xls")


In [29]:
d=pd.read_excel("zamato_data/uploadzamato.xls")

In [30]:
len(d.values[0])

23